In [ ]:
#𝐕𝐢𝐫𝐭𝐮𝐚𝐥 𝐄𝐱𝐡𝐢𝐛𝐢𝐭𝐢𝐨𝐧 𝐆𝐞𝐧𝐞𝐫𝐚𝐭𝐨𝐫

#In this project, I aimed to create a custom immersive exhibition generator using Spotify API and the Art Institute of Chicago's API. 
#The idea is to create an app where you can chose what genre of art you want to see and a playlist is automatically created to accompany
#the art to elevate your viewing experience.

In [ ]:
#Importing necessary tools

import spotipy
import spotipy.util as util
import json
import webbrowser
import pandas as pd
import urllib.request

In [ ]:
#Accessing Spotify API 

cred = "spotify_keys.json"
with open(cred,"r") as key_file:
    api_tokens = json.load(key_file)

key_file.close()

In [ ]:
client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username'] 

In [ ]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirectURI)

In [ ]:
sp = spotipy.Spotify(auth=token)

In [ ]:
#Checking Spotify recommended genres so that I know what is available.
genre_seeds = sp.recommendation_genre_seeds()

In [ ]:
genre_seeds

In [ ]:
#On the Art Institude of Chicago Github, I found a CSV file with their top 200 art pieces (they claim its 200, but there is way more). 
# I wanted to start working with this data to see how they categories the archive and what this means for me. 

art_data = pd.read_csv("AIC_data.csv")

In [ ]:
display(art_data)

In [ ]:
#That's quite a lot of data! I then wanted to see all the name of the departments/types of art that are in the data set.

departments = art_data["department_title"].unique()

print("Department Titles",departments)

In [ ]:
#At this point, the next logical step is to connect to the main API and see what is availble there. 

url = "https://api.artic.edu/api/v1/artworks?fields=id,title,artist_display,date_display,classification_titles,department_title,image_id"
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
all_art_data= json.loads(response.read())
all_art_data

In [ ]:
#Making sense of the information above.

for artwork in all_art_data['data'][:20]: 
    
    print("Title:", artwork['title'])
    print("Artist:", artwork.get('artist_display',))
    print("Date:", artwork.get('date_display'))
    print("Classification:", artwork.get('classification_titles'))
    print("Department:", artwork.get('department_title'))
    print("\n")

In [ ]:
#What are the unique department titles in this data set?
art_departments = [department_titles['department_title'] for department_titles in all_art_data['data']]
art_departments

In [ ]:
#Here I wanted to combine the data I got from the CSV and what I'm getting straight from the API so I'm not missing anything.

api_art_data = pd.DataFrame(data['data'])

csv_art_data = pd.read_csv("AIC_data.csv")

combined_art_data = pd.concat([csv_art_data, api_art_data], ignore_index=True)

In [ ]:
#Checking all the departments and all the classifications in my combined data set.

all_departments = combined_art_data["department_title"].unique()
print("All Departments:", all_departments)

all_classifications = combined_art_data["classification_titles"].explode().unique()
print("\nAll Classifications:", all_classifications)

In [ ]:
#From what I found, I was only interested in specific departments and wanted to filter out the rest. 

selected_departments = ["Modern Art", "Contemporary Art", "Photography and Media", "Prints and Drawings", "Textiles"]

department_filter = combined_art_data["department_title"].isin(selected_departments)

filtered_art_data = combined_art_data[department_filter]

In [ ]:
filtered_art_data

In [ ]:
artwork_count = len(filtered_art_data)
artwork_count

In [ ]:
#Checking how many pieces are in each department.

department_counts = filtered_art_data["department_title"].value_counts()

department_counts

In [ ]:
#Getting just the titles in a list, out of curiosity.
titles = filtered_art_data["title"].tolist()
for title in titles:
    print(title)

In [ ]:
# Art-to-Music Mapping Concept

# Modern Art
# Genre Seeds: ambient, experimental, trip-hop, electronic, idm
# Vibe: Abstract, innovative, and textured. These genres offer an atmospheric, boundary-pushing sound without being too familiar, 
# making them a perfect match for modern art that often explores the abstract and unconventional.

# Contemporary Art
# Genre Seeds: downtempo, synth-pop, chill, minimal-techno, new-age
# Vibe: Smooth and versatile, ideal for contemplative and bold contemporary pieces. These genres help create a modern, reflective atmosphere,
# supporting the dynamic and introspective qualities often found in contemporary art.

# Photography and Media
# Genre Seeds: dark-ambient, electronic, lo-fi, cinematic, ambient
# Vibe: Emotive and moody, with genres that add a cinematic or storytelling feel. These choices enhance the evocative quality of photography and media, 
# which often capture stories or moments frozen in time.

# Prints and Drawings
# Genre Seeds: ethereal, ambient, downtempo, soundscapes, neo-classical
# Vibe: Delicate and intricate, these genres enhance the tactile feel of prints and drawings. The music is intended to be immersive yet subtle, 
# echoing the careful details and craftsmanship of print and drawing techniques.

# Textiles
# Genre Seeds: world-music, new-age, ethereal, drone, ambient
# Vibe: Rooted yet expansive, offering a cultural or meditative quality without being overly traditional. These genres are intended to evoke a 
# connection to nature and tradition, while still feeling fresh and expansive for textile art.

In [ ]:
#Mapping what style of art should be matched with what type of art.

art_to_music_map = {
    "Modern Art": ["ambient", "experimental", "trip-hop", "electronic", "idm"],
    "Contemporary Art": ["downtempo", "synth-pop", "chill", "minimal-techno", "new-age"],
    "Photography and Media": ["dark-ambient", "electronic", "lo-fi", "cinematic", "ambient"],
    "Prints and Drawings": ["ethereal", "ambient", "downtempo", "soundscapes", "neo-classical"],
    "Textiles": ["new-age", "ethereal", "drone", "ambient"]
}

In [ ]:
#Creating randomised song recommendations and checking to see if mapping worked.

for art_category, genre_seeds in art_to_music_map.items():
    print(f"\nMusic Recommendations for {art_category}:")

    recommendations = sp.recommendations(seed_genres=genre_seeds, limit=5)

    for track in recommendations['tracks']:
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        print(f" ♫ {track_name} by {artist_name}")

In [ ]:
# At this point, I wanted to bring in images of the art pieces in my chosen categories so I can start creating randomised mixtures of the art
# and music. I know from the AIC API that they provide URLs for images and jupyter is able to show images. However, I realised at this point
# that in the CSV data set, there is no column for Image URLs. I tired getting all the image ids from my art data set and see if there is a 
# way for me to get URLs using those image ids. Unfortunately, that become too confusing and convoluted for me to understand so I pivioted 
# to collecting images straight from the API, still using my parameters. 

In [ ]:
# Below is the code for the Virutal Exhibiton Generator. First step is making it possible to import images into Jupyter Notebook.
# Getting the images and filtering only for the categories I want. 

In [ ]:
#Importing images into Jupyter Notebook
from IPython.display import display, Image


def create_virtual_exhibition(selected_categories, num_images=3, num_songs=5):
    track_uris = []
    print(f"Virtual Exhibiton of {selected_categories}\n")
    
    for category in selected_categories:
        print(f"\nExhibition Category: {category}")

        
# Getting artworks in the chosen category
        artworks_in_category = artworks_df[artworks_df["department_title"] == category]

        
# Randomly select images from the category
        sample_artworks = artworks_in_category.sample(n=min(num_images, len(artworks_in_category)))
        for _, artwork in sample_artworks.iterrows():
            print(f"Title: {artwork['title']} - Artist: {artwork.get('artist_title', 'Unknown')}")
            image_url = artwork.get("image_url")
            if pd.notna(image_url) and image_url:
                display(Image(url=image_url, width=300))

# Getting music recommendations based on the genre seeds for each category
        
        genre_seeds = art_to_music_map.get(category, [])
        recommendations = sp.recommendations(seed_genres=genre_seeds, limit=num_songs)
        for track in recommendations['tracks']:
            track_name = track['name']
            artist_name = track['artists'][0]['name']
            print(f" - {track_name} by {artist_name}")
            track_uris.append(track['uri'])

    
# Create a Spotify playlist with the collected track URIs
    
    playlist = sp.user_playlist_create(user=username, name="Virtual Gallery Playlist", public=True)
    sp.playlist_add_items(playlist_id=playlist['id'], items=track_uris)
    print("\nSpotify Playlist Link:", playlist['external_urls']['spotify'])

In [ ]:
#By selecting what categories you want to see, how many number of images and songs you want, a mini "exhibiton" will be generated for you.

selected_categories = ["Modern Art", "Contemporary Art"]
create_virtual_exhibition(selected_categories, num_images=5, num_songs=5)

In [ ]:
selected_categories = ["Photography and Media", "Prints and Drawings"]
create_virtual_exhibition(selected_categories, num_images=5, num_songs=5)